In [1]:
dbutils.library.installPyPI('tpot')

Out[1]: True

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupShuffleSplit
from tpot import TPOTClassifier

/local_disk0/pythonVirtualEnvDirs/virtualEnv-11575ceb-b974-45a9-8a45-cd2ba174301e/lib/python3.7/site-packages/tpot/builtins/__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
 warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")

In [3]:
file_path = "/FileStore/Tables/Historical/signals"
signals_df = spark.read.parquet(file_path).toPandas()

In [4]:
columns = ['signal','sma10','ema20','ema30','price','high','low','open','volume','symbol']
features = ['sma10','ema20','ema30','price','high','low','open','volume','symbol']
target = ['signal']

data = signals_df[columns].copy().dropna()
df = data[columns]
labels = data[target]
# df = df[df["sma10"].notnull()]
# df = df[df["ema20"].notnull()]
# df = df[df["ema30"].notnull()]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 312142 entries, 0 to 315666
Data columns (total 10 columns):
signal 312142 non-null float64
sma10 312142 non-null float64
ema20 312142 non-null float64
ema30 312142 non-null float64
price 312142 non-null float64
high 312142 non-null float64
low 312142 non-null float64
open 312142 non-null float64
volume 312142 non-null float64
symbol 312142 non-null object
dtypes: float64(9), object(1)
memory usage: 26.2+ MB

In [6]:
X = StandardScaler().fit_transform(df.iloc[:, :-1].values)
y = labels.iloc[:, -1].values
groups = df['symbol'].values

In [7]:
train_idx, test_idx = next(GroupShuffleSplit().split(X, y, groups))
X_train, X_test, y_train, y_test = X[train_idx], X[test_idx], y[train_idx], y[test_idx]
groups_train, groups_test = groups[train_idx], groups[test_idx]

In [8]:
tpot = TPOTClassifier(generations=5, population_size=50, scoring='balanced_accuracy', verbosity=2)
tpot.fit(X_train, y_train, groups=groups_train)
print(tpot.score(X_test, y_test))

Warning: xgboost.XGBClassifier is not available and will not be used by TPOT.
Optimization Progress: 0%| | 0/300 [00:00<?, ?pipeline/s]Optimization Progress: 1%| | 2/300 [00:07<17:30, 3.53s/pipeline]Optimization Progress: 1%| | 3/300 [00:16<26:09, 5.29s/pipeline]Optimization Progress: 1%|▏ | 4/300 [00:47<1:04:18, 13.04s/pipeline]Optimization Progress: 2%|▏ | 5/300 [04:10<5:43:36, 69.89s/pipeline]Optimization Progress: 2%|▏ | 6/300 [05:38<6:09:32, 75.42s/pipeline]Optimization Progress: 2%|▏ | 7/300 [11:39<13:07:25, 161.25s/pipeline]Optimization Progress: 3%|▎ | 9/300 [12:16<9:34:12, 118.39s/pipeline] Optimization Progress: 3%|▎ | 10/300 [12:24<6:51:21, 85.11s/pipeline]Optimization Progress: 4%|▎ | 11/300 [13:45<6:45:08, 84.11s/pipeline]Optimization Progress: 4%|▍ | 12/300 [18:46<11:54:59, 148.96s/pipeline]Optimization Progress: 5%|▍ | 14/300 [19:18<8:40:19, 109.16s/pipeline] Optimization Progress: 5%|▌ | 15/300 [19:34<6:24:58, 81.05s/pipeline] Optimization Progress: 5%|▌ | 16/300 [19:44<4:42:38, 59.71s/pipeline]Optimization Progress: 6%|▌ | 17/300 [19:56<3:34:48, 45.54s/pipeline]Optimization Progress: 6%|▌ | 18/300 [20:04<2:40:25, 34.13s/pipeline]Optimization Progress: 6%|▋ | 19/300 [20:12<2:03:30, 26.37s/pipeline]Optimization Progress: 7%|▋ | 20/300 [20:52<2:21:45, 30.38s/pipeline]Optimization Progress: 7%|▋ | 21/300 [25:52<8:37:23, 111.27s/pipeline]Optimization Progress: 8%|▊ | 23/300 [25:59<6:04:31, 78.96s/pipeline] Optimization Progress: 8%|▊ | 24/300 [26:06<4:24:29, 57.50s/pipeline]Optimization Progress: 8%|▊ | 25/300 [26:21<3:24:47, 44.68s/pipeline]Optimization Progress: 9%|▊ | 26/300 [26:42<2:51:54, 37.64s/pipeline]Optimization Progress: 9%|▉ | 27/300 [28:46<4:48:08, 63.33s/pipeline]Optimization Progress: 9%|▉ | 28/300 [29:26<4:15:34, 56.38s/pipeline]Optimization Progress: 10%|▉ | 29/300 [34:26<9:44:45, 129.47s/pipeline]Optimization Progress: 10%|█ | 31/300 [35:25<7:26:24, 99.57s/pipeline] Optimization Progress: 11%|█ | 32/300 [35:33<5:22:09, 72.13s/pipeline]Optimization Progress: 11%|█ | 33/300 [35:42<3:55:54, 53.01s/pipeline]Optimization Progress: 11%|█▏ | 34/300 [36:17<3:31:56, 47.81s/pipeline]Optimization Progress: 12%|█▏ | 35/300 [36:31<2:45:33, 37.48s/pipeline]Optimization Progress: 12%|█▏ | 36/300 [37:29<3:11:44, 43.58s/pipeline]Optimization Progress: 12%|█▏ | 37/300 [40:02<5:34:55, 76.41s/pipeline]Optimization Progress: 13%|█▎ | 38/300 [40:10<4:04:54, 56.08s/pipeline]Optimization Progress: 13%|█▎ | 39/300 [42:18<5:37:25, 77.57s/pipeline]Optimization Progress: 13%|█▎ | 40/300 [42:26<4:05:29, 56.65s/pipeline]Optimization Progress: 14%|█▎ | 41/300 [42:34<3:01:44, 42.10s/pipeline]Optimization Progress: 14%|█▍ | 42/300 [42:41<2:16:09, 31.66s/pipeline]Optimization Progress: 14%|█▍ | 43/300 [42:51<1:47:04, 25.00s/pipeline]Optimization Progress: 15%|█▍ | 44/300 [45:48<5:01:26, 70.65s/pipeline]Optimization Progress: 15%|█▌ | 45/300 [47:41<5:53:48, 83.25s/pipeline]Optimization Progress: 15%|█▌ | 46/300 [48:14<4:49:27, 68.38s/pipeline]Optimization Progress: 16%|█▌ | 47/300 [48:21<3:30:56, 50.03s/pipeline]Optimization Progress: 16%|█▌ | 48/300 [48:29<2:37:02, 37.39s/pipeline]Optimization Progress: 16%|█▋ | 49/300 [49:38<3:15:24, 46.71s/pipeline]Optimization Progress: 17%|█▋ | 50/300 [49:46<2:26:41, 35.21s/pipeline]Optimization Progress: 17%|█▋ | 51/300 [51:38<4:01:20, 58.16s/pipeline]Optimization Progress: 17%|█▋ | 52/300 [53:11<4:44:14, 68.77s/pipeline]Optimization Progress: 18%|█▊ | 53/300 [53:53<4:09:47, 60.68s/pipeline]Optimization Progress: 18%|█▊ | 54/300 [56:55<6:37:34, 96.97s/pipeline]Optimization Progress: 19%|█▉ | 58/300 [56:57<4:34:23, 68.03s/pipeline]Optimization Progress: 20%|█▉ | 59/300 [57:17<3:35:23, 53.63s/pipeline]Optimization Progress: 20%|██ | 60/300 [57:26<2:40:29, 40.12s/pipeline]Optimization Progress: 20%|██ | 61/300 [58:20<2:57:09, 44.48s/pipeline]Optimization Progress: 21%|██ | 62/300 [58:28<2:12:30, 33.40s/pipeline]Optimization Progress: 21%|██

In [9]:
tpot.export('prices_pipeline.py')

In [10]:
print(tpot.export())

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# NOTE: Make sure that the outcome column is labeled 'target' in the data file
tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
features = tpot_data.drop('target', axis=1)
training_features, testing_features, training_target, testing_target = \
 train_test_split(features, tpot_data['target'], random_state=None)

# Average CV score on the training set was: 1.0
exported_pipeline = RandomForestClassifier(bootstrap=True, criterion="gini", max_features=0.6500000000000001, min_samples_leaf=10, min_samples_split=16, n_estimators=100)

exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)